# Gallery Example: Hyper/Hyper/1 Linear Network

This example demonstrates a linear network with n queues in series:
- **Arrivals**: Hyperexponential distribution
- **Service**: Hyperexponential with varying parameters per queue
- **Servers**: 1 server per queue
- **Scheduling**: FCFS
- **Topology**: Linear (tandem) network

The service times vary across queues to create a 'pyramid' utilization pattern.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_hyphyp1_linear(n=2, Umax=0.9):
    """Create Hyp/Hyp/1-Linear queueing network"""
    model = Network('Hyp/Hyp/1-Linear')
    
    # Block 1: nodes
    nodes = []
    source = Source(model, 'mySource')
    nodes.append(source)
    
    for i in range(n):
        queue = Queue(model, f'Queue{i+1}', SchedStrategy.FCFS)
        nodes.append(queue)
    
    sink = Sink(model, 'mySink')
    nodes.append(sink)
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    
    # HyperExp arrivals
    source.set_arrival(oclass, HyperExp.fit_mean_and_scv(1, 2))
    
    # Create pyramid utilization pattern
    half = n // 2
    means = list(np.linspace(0.1, Umax, half))
    if n % 2 == 0:
        means = means + means[::-1]
    else:
        means = means + [Umax] + means[::-1]
    
    # Set service for each queue with increasing SCV
    for i in range(n):
        nodes[i+1].set_service(oclass, HyperExp.fit_mean_and_scv(means[i], 1 + i + 1))
    
    # Block 3: topology
    model.link(Network.serial_routing(*nodes))
    
    return model

# Create the model with 2 queues
model = gallery_hyphyp1_linear(n=2)

## About Linear (Tandem) Networks

In a linear network:
- Jobs pass through all queues in sequence
- Each queue has different service characteristics
- Bottleneck determines overall throughput

This example uses:
- 'Pyramid' mean service times: increasing then decreasing
- Increasing SCV: more variable service at later queues
- High variability in both arrivals and service (HyperExp)

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)